In [ ]:
!pip install langchain wikipedia transformers


In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.utilities import WikipediaAPIWrapper
from transformers import pipeline

hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-small", max_new_tokens=128)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

wiki = WikipediaAPIWrapper()


In [ ]:
def wiki_search(query: str) -> str:
    try:
        result = wiki.run(query)
        if not result.strip():
            return "No relevant info found on Wikipedia."
        return result
    except Exception as e:
        return f"Error fetching Wikipedia data: {e}"


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this text concisely:\n\n{text}"
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

def summarize_text(text: str) -> str:
    try:
        return summary_chain.invoke({"text": text})["text"]
    except Exception as e:
        return f"Could not summarize due to error: {e}"


In [ ]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="Summarize this text concisely:\n\n{text}"
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

def summarize_text(text: str) -> str:
    try:
        return summary_chain.invoke({"text": text})["text"]
    except Exception as e:
        return f"Could not summarize due to error: {e}"


In [ ]:
from textwrap import wrap

def chunk_text(text, max_chars=1500):
    return wrap(text, max_chars)

query = "Explain quantum computing in simple terms."

# Step 1: Search Wikipedia
wiki_result = wiki_search(query)
print("Wikipedia Result (preview):\n", wiki_result[:500], "...\n")  # preview first 500 chars

# Step 2: If text is short, summarize directly, else chunk & summarize each
if len(wiki_result) < 1500:
    final_answer = summarize_text(wiki_result)
else:
    chunks = chunk_text(wiki_result, max_chars=1500)[:2]  # limit to first 2 chunks for speed
    summaries = []
    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        summaries.append(summarize_text(chunk))
    final_answer = " ".join(summaries)

print("\nFinal Answer:\n", final_answer)


In [ ]:
!pip install gradio --quiet
import gradio as gr

def chat_agent(query):
    wiki_result = wiki_search(query)
    if len(wiki_result) < 1500:
        return summarize_text(wiki_result)
    else:
        chunks = chunk_text(wiki_result, max_chars=1500)[:2]
        summaries = [summarize_text(chunk) for chunk in chunks]
        return " ".join(summaries)

iface = gr.Interface(
    fn=chat_agent,
    inputs=gr.Textbox(lines=2, placeholder="Ask me anything..."),
    outputs="text",
    title="LangChain Agent Prototype",
    description="Ask questions, get answers from Wikipedia + summarization"
)

iface.launch()
